# ব্যয় দাবি বিশ্লেষণ

এই নোটবুকটি দেখায় কীভাবে প্লাগইন ব্যবহার করে এজেন্ট তৈরি করা যায় যা স্থানীয় রসিদের ছবি থেকে ভ্রমণ ব্যয় প্রক্রিয়া করে, একটি ব্যয় দাবি ইমেইল তৈরি করে এবং পাই চার্ট ব্যবহার করে ব্যয় ডেটা চিত্রিত করে। এজেন্টরা কাজের প্রসঙ্গ অনুযায়ী ফাংশনগুলি গতিশীলভাবে নির্বাচন করে।

ধাপসমূহ:
1. OCR এজেন্ট স্থানীয় রসিদের ছবি প্রক্রিয়া করে এবং ভ্রমণ ব্যয়ের ডেটা বের করে।
2. ইমেইল এজেন্ট একটি ব্যয় দাবি ইমেইল তৈরি করে।

### ভ্রমণ ব্যয়ের একটি উদাহরণ:
ধরুন আপনি একজন কর্মচারী যিনি ব্যবসায়িক মিটিংয়ের জন্য অন্য শহরে ভ্রমণ করছেন। আপনার কোম্পানির নীতি অনুযায়ী সমস্ত যুক্তিসঙ্গত ভ্রমণ-সম্পর্কিত ব্যয় ফেরত দেওয়া হয়। এখানে সম্ভাব্য ভ্রমণ ব্যয়ের একটি বিবরণ দেওয়া হলো:
- পরিবহন:
আপনার বাড়ির শহর থেকে গন্তব্য শহরে যাতায়াতের জন্য বিমানের টিকিট।
এয়ারপোর্টে যাতায়াতের জন্য ট্যাক্সি বা রাইড-হেইলিং পরিষেবা।
গন্তব্য শহরে স্থানীয় পরিবহন (যেমন পাবলিক ট্রানজিট, ভাড়ার গাড়ি বা ট্যাক্সি)।

- আবাসন:
মিটিং ভেন্যুর কাছাকাছি একটি মধ্য-পর্যায়ের ব্যবসায়িক হোটেলে তিন রাতের থাকার ব্যবস্থা।

- খাবার:
কোম্পানির দৈনিক ভাতা নীতির ভিত্তিতে প্রাতঃরাশ, মধ্যাহ্নভোজ এবং রাতের খাবারের জন্য দৈনিক খাবারের ভাতা।

- অন্যান্য খরচ:
এয়ারপোর্টে পার্কিং ফি।
হোটেলে ইন্টারনেট ব্যবহারের চার্জ।
টিপস বা ছোট সার্ভিস চার্জ।

- ডকুমেন্টেশন:
আপনি সমস্ত রসিদ (ফ্লাইট, ট্যাক্সি, হোটেল, খাবার ইত্যাদি) এবং একটি সম্পূর্ণ ব্যয় রিপোর্ট জমা দেন ফেরতের জন্য।


## প্রয়োজনীয় লাইব্রেরি আমদানি করুন

নোটবুকের জন্য প্রয়োজনীয় লাইব্রেরি এবং মডিউলগুলি আমদানি করুন।


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## ব্যয় মডেল সংজ্ঞায়িত করুন

 পৃথক ব্যয়ের জন্য একটি Pydantic মডেল তৈরি করুন এবং একটি ExpenseFormatter ক্লাস তৈরি করুন যা ব্যবহারকারীর প্রশ্নকে কাঠামোবদ্ধ ব্যয় ডেটায় রূপান্তর করবে।

 প্রতিটি ব্যয় নিম্নলিখিত ফরম্যাটে উপস্থাপিত হবে:
 `{'date': '07-Mar-2025', 'description': 'গন্তব্যে ফ্লাইট', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## এজেন্ট সংজ্ঞায়িত করা - ইমেইল তৈরি করা

একটি এজেন্ট ক্লাস তৈরি করুন যা খরচ দাবি জমা দেওয়ার জন্য একটি ইমেইল তৈরি করে।
- এই এজেন্ট `kernel_function` ডেকোরেটর ব্যবহার করে একটি ফাংশন সংজ্ঞায়িত করে যা খরচ দাবি জমা দেওয়ার জন্য একটি ইমেইল তৈরি করে।
- এটি খরচের মোট পরিমাণ গণনা করে এবং ইমেইলের বডিতে বিস্তারিত ফরম্যাট করে।


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# রসিদের ছবি থেকে ভ্রমণ খরচ বের করার জন্য এজেন্ট

রসিদের ছবি থেকে ভ্রমণ খরচ বের করার জন্য একটি এজেন্ট ক্লাস তৈরি করুন।  
- এই এজেন্ট `kernel_function` ডেকোরেটর ব্যবহার করে একটি ফাংশন সংজ্ঞায়িত করে, যা রসিদের ছবি থেকে ভ্রমণ খরচ বের করে।  
- রসিদের ছবিকে টেক্সটে রূপান্তর করতে OCR (অপটিক্যাল ক্যারেক্টার রিকগনিশন) ব্যবহার করুন এবং প্রাসঙ্গিক তথ্য যেমন তারিখ, বিবরণ, পরিমাণ, এবং বিভাগ বের করুন।  


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## খরচ প্রক্রিয়াকরণ

খরচ প্রক্রিয়াকরণের জন্য একটি অ্যাসিঙ্ক্রোনাস ফাংশন সংজ্ঞায়িত করুন, যা প্রয়োজনীয় এজেন্ট তৈরি এবং নিবন্ধন করে এবং তারপর তাদের আহ্বান করে।  
- এই ফাংশন পরিবেশ ভেরিয়েবল লোড করে, প্রয়োজনীয় এজেন্ট তৈরি করে এবং তাদের প্লাগইন হিসেবে নিবন্ধন করে খরচ প্রক্রিয়াকরণ সম্পন্ন করে।  
- এটি দুটি এজেন্টের সাথে একটি গ্রুপ চ্যাট তৈরি করে এবং খরচের ডেটার উপর ভিত্তি করে ইমেইল এবং পাই চার্ট তৈরি করার জন্য একটি প্রম্পট বার্তা পাঠায়।  
- এটি চ্যাট আহ্বানের সময় ঘটে যাওয়া যেকোনো ত্রুটি পরিচালনা করে এবং এজেন্টগুলোর সঠিক পরিষ্কার-পরিচ্ছন্নতা নিশ্চিত করে।  


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## প্রধান ফাংশন

মূল ফাংশনটি সংজ্ঞায়িত করুন যাতে কনসোল পরিষ্কার করা যায় এবং `process_expenses` ফাংশনটি অ্যাসিঙ্ক্রোনাসভাবে চালানো যায়।


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় লেখা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হয়। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
